In [1]:
import os

In [2]:
%pwd

'd:\\End-to-end-Machine-Learning-Project\\research'

In [3]:
# Move to project directory
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-end-Machine-Learning-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str
    

In [6]:
# Configuration manager temp Libraries
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [7]:
# Setting up configuration manager model trainer 
class ConfigurationManager:
    def __init__ (
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        #schema = self.schema.TARGET_COLUMN

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = params.target_column
        )
        return model_trainer_config




In [8]:
# Define componets dir
import os
import pandas as pd
import joblib
from mlProject import logger
from sklearn.linear_model import ElasticNet



In [ ]:
# Implementing model trainer componentclass ModelTrainer:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        logger.info("Loading training data")
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        logger.info("Training model")
        model = ElasticNet(
            alpha=self.config.alpha,
            l1_ratio=self.config.l1_ratio,
            random_state=42
        )

        X_train = train_data.drop(columns=[self.config.target_column])
        X_test = test_data.drop(columns=[self.config.target_column])
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        model.fit(X_train, y_train)


        # Save model
        logger.info("Saving model")
        model_file_path = os.path.join(self.config.root_dir, self.config.model_name)
        
        # Create directory
        try:
            os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
            joblib.dump(model, model_file_path)
        except Exception as e:
            logger.error(f"Failed to save model: {str(e)}")
            raise
        
        logger.info(f"Model saved to {model_file_path}")

       

In [10]:
# Setting up the model trainer pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e


[2025-07-02 15:15:25,486: INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-07-02 15:15:25,491: INFO: common: YAML file params.yaml loaded successfully.]


[2025-07-02 15:15:25,498: INFO: common: YAML file schema.yaml loaded successfully.]
[2025-07-02 15:15:25,502: INFO: common: Created directory: artifacts]
[2025-07-02 15:15:25,506: INFO: 2906954383: Loading training data]
[2025-07-02 15:15:25,523: INFO: 2906954383: Training model]
[2025-07-02 15:15:25,538: INFO: 2906954383: Saving model]
[2025-07-02 15:15:25,552: INFO: 2906954383: Model saved to artifacts/model_trainer\model.joblib]
